In [379]:
import json

with open('timezones.json') as f:
    timezones = json.load(f)
    
print(timezones)

#indian standard time
def isTimezoneName(name):
    for timezone in timezones:
        if(timezone["value"] == name):
            return True
    return False

#indian standard time -> ist
# ist -> ist
def getTimezone(name):
    modified = name[:-1]
    for timezone in timezones:
        if(timezone["value"] == name):
            return timezone["abbr"]
        elif(timezone["abbr"] == name or timezone["abbr"] == modified):
            return name
    return False

#ist is a timezone or not 
def isTimezoneAbbr(abbr):
    modified = abbr[:-1]
    for timezone in timezones:
        if(timezone["abbr"] == abbr or timezone["abbr"] == modified):
            return True
    return False

def isTimezoneAbbrOrig(abbr):
    for timezone in timezones:
        if(timezone["abbr"] == abbr):
            return True
    return False

#isTimezoneAbbr("IST.")


[{'value': 'Dateline Standard Time', 'abbr': 'DST', 'utc': ['Etc/GMT+12']}, {'value': 'UTC-11', 'abbr': 'U', 'utc': ['Etc/GMT+11', 'Pacific/Midway', 'Pacific/Niue', 'Pacific/Pago_Pago']}, {'value': 'Hawaiian Standard Time', 'abbr': 'HST', 'utc': ['Etc/GMT+10', 'Pacific/Honolulu', 'Pacific/Johnston', 'Pacific/Rarotonga', 'Pacific/Tahiti']}, {'value': 'Alaskan Standard Time', 'abbr': 'AKDT', 'utc': ['America/Anchorage', 'America/Juneau', 'America/Nome', 'America/Sitka', 'America/Yakutat']}, {'value': 'Pacific Standard Time (Mexico)', 'abbr': 'PDT', 'utc': ['America/Santa_Isabel']}, {'value': 'Pacific Daylight Time', 'abbr': 'PDT', 'utc': ['America/Dawson', 'America/Los_Angeles', 'America/Tijuana', 'America/Vancouver', 'America/Whitehorse']}, {'value': 'Pacific Standard Time', 'abbr': 'PST', 'utc': ['America/Dawson', 'America/Los_Angeles', 'America/Tijuana', 'America/Vancouver', 'America/Whitehorse', 'PST8PDT']}, {'value': 'US Mountain Standard Time', 'abbr': 'UMST', 'utc': ['America/Cres

In [375]:
import os
#print(os.getcwd())

location = os.path.realpath(
    os.path.join(os.getcwd(), os.path.dirname("111703070_Assgn1Dataset-1.txt")))
#print(location)

In [362]:
filePath = "111703070_Assgn1Dataset-1.txt"

try:
    with open(filePath, 'r') as fp:
        tweets = fp.readlines()
except IOError as e:
    print("Couldn't open or read file (%s)." % e)

In [363]:
months = {
    "January":"01",
    "February":"02",
    "March":"03",
    "April":"04",
    "May":"05",
    "June":"06",
    "July":"07",
    "August":"08",
    "September":"09",
    "October":"10",
    "November":"11",
    "December":"12"
}

In [364]:
def isNumberinWord(word):
    for letter in word:
        if(letter.isdigit()):
            return True
    return False

#14-07- == -07-14
def reverseDate(date):
    tempDate = ""
    i = len(date) - 1
    stack = []
    while i >= 0:
        if(date[i] == "-"):
            while(len(stack) != 0):
                tempDate += stack.pop()
            tempDate += date[i]
        else:
            stack.append(date[i])
        i = i - 1
    while(len(stack) != 0):
        tempDate += stack.pop()
    return tempDate
        
# def convertDate(dateRec):
#     date = isDate(dateRec)
#     if(date == None):
#         prefix = "CF:D:"
#         if(len(finalDate) == 2):
#             date = prefix+"????-"+finalDate
#         elif(len(finalDate) == 6):
#             date = prefix+"????"+ reverseDate(finalDate)
#         else:
#             date = prefix+ reverseDate(finalDate)
#     if(date == "CF:D:"):
#         return None
#     return date

def convertDate(dateRec):
    date = isDate(dateRec)
    if(date == None):
        prefix = "CF:D:"
        if(len(dateRec) == 2):
            date = prefix+"????-"+dateRec
        elif(len(dateRec) == 6):
            date = prefix+"????"+ reverseDate(dateRec)
        else:
            date = prefix+ reverseDate(dateRec)
    if(date == "CF:D:"):
        return None
    return date

def isFinalDate(date):
    if(date.startswith("CF:D:")):
        return True
    else:
        return False

In [365]:
import re

tokens = []
for tweet in tweets :
    
    words = tweet.split()
    
    wordIndex = 0
    while (wordIndex < len(words)) :
        if(wordIndex + 1 < len(words) and (words[wordIndex+1] == "o'clock" or words[wordIndex+1] == "O'Clock")):
            if(isTimeFormat(words[wordIndex] + " am")):
                words[wordIndex+1] = "am"
                
        #7:00:00 || 7:00|| 7 && AM||PM && IST||Indian Standard Time||any word after it
        if(wordIndex+1 < len(words)  and
            (words[wordIndex+1] == "AM" or words[wordIndex+1] == "PM" or
            words[wordIndex+1] == "am" or words[wordIndex+1] == "pm")
           and isTimeFormat(words[wordIndex]+" " + words[wordIndex+1]) and wordIndex+2 < len(words)):
                time = isTime(words[wordIndex]+" "+words[wordIndex+1])
                if(isTimezoneAbbr(words[wordIndex+2])):
                    if(isTimezoneAbbrOrig(words[wordIndex+2])):
                        time += ":" + getTimezone(words[wordIndex+2])
                        words.pop(wordIndex) #IST
                        words.pop(wordIndex) #am/pm
                        words.pop(wordIndex) #number
                        words.insert(wordIndex, time)
                    else:
                        punc = words[wordIndex+2][-1]
                        time += ":" + getTimezone(words[wordIndex+2][:-1])
                        words.pop(wordIndex) #IST.
                        words.pop(wordIndex)
                        words.pop(wordIndex)
                        words.insert(wordIndex, time)
                        words.insert(wordIndex+1, punc)
                else:
                    tempStr = words[wordIndex+2] #indian
                    j = wordIndex+3 #standard
                    countWords = 1
                    puncFlag = 0
                    while(j < len(words)):
                        if(re.findall("[\.!?:,;\'\"\)\(\]-]$", words[j])):
                            puncFlag = 1
                            punc = words[j][-1]
                            tempStr += " " + words[j][:-1]
                        else:
                            tempStr += " " + words[j]
                        countWords += 1
                        if(words[j] == "Time" or words[j] == "time" or words[j][:-1] == "Time" or words[j][:-1] == "time"):
                            break
                        j += 1
                    if(isTimezoneName(tempStr)):
                        time += ":" + getTimezone(tempStr)
                        while(countWords > 0):
                            words.pop(wordIndex)
                            countWords -= 1
                        words.pop(wordIndex) #am or pm
                        words.pop(wordIndex) #number
                        words.insert(wordIndex, time)
                        if(puncFlag == 1):
                            words.insert(wordIndex+1, punc)
                    else:
                        time = isTime(words[wordIndex]+" "+words[wordIndex+1])
                        words.pop(wordIndex)
                        words.pop(wordIndex)
                        words.insert(wordIndex, time)
            
        #7:00:00 || 7:00  IST || 7:00 IST.
        elif(isTimeFormat(words[wordIndex]) and 
           wordIndex+1 < len(words) and isTimezoneAbbr(words[wordIndex+1])):
            time = isTime(words[wordIndex])
            if(isTimezoneAbbrOrig(words[wordIndex+1])):
                time += ":" + getTimezone(words[wordIndex+1])
                words.pop(wordIndex)
                words.pop(wordIndex)
                words.insert(wordIndex, time)
            else:
                #IST.
                punc = words[wordIndex+1][-1]
                time += ":" + getTimezone(words[wordIndex+1][:-1])
                words.pop(wordIndex)
                words.pop(wordIndex)
                words.insert(wordIndex, time)
                words.insert(wordIndex+1, punc)
            
        # 7:00 Indian standard time || 7:00 Indian standard time.
        elif(isTimeFormat(words[wordIndex]) and wordIndex+1 < len(words) and 
             (words[wordIndex+1] != "AM" or words[wordIndex+1] != "PM" or
            words[wordIndex+1] != "am" or words[wordIndex+1] != "pm")):
            time = isTime(words[wordIndex])
            tempStr = words[wordIndex+1] #indian
            j = wordIndex+2 #standard
            countWords = 1
            puncFlag = 0
            while(j < len(words)):
                if(re.findall("[\.!?:,;\'\"\)\(\]-]$", words[j])):
                    puncFlag = 1
                    punc = words[j][-1]
                    tempStr += " " + words[j][:-1]
                else:
                    tempStr += " " + words[j]
                countWords += 1
                if(words[j] == "Time" or words[j] == "time" or words[j][:-1] == "Time" or words[j][:-1] == "time"):
                    break
                j += 1
            if(isTimezoneName(tempStr)):
                time += ":" + getTimezone(tempStr)
                while(countWords > 0):
                    words.pop(wordIndex)
                    countWords -= 1
                words.pop(wordIndex) #number
                words.insert(wordIndex, time)
                if(puncFlag == 1):
                    words.insert(wordIndex+1, punc)
            else:
                time = isTime(words[wordIndex])
                words.pop(wordIndex)
                words.insert(wordIndex, time)
        
        #7:00:00 || 7:00|| 7 AM||PM
        elif(wordIndex+1 < len(words)  and
            (words[wordIndex+1] == "AM" or words[wordIndex+1] == "PM" or
            words[wordIndex+1] == "am" or words[wordIndex+1] == "pm")
            and isTimeFormat(words[wordIndex]+" " + words[wordIndex+1])):
            time = isTime(words[wordIndex]+" "+words[wordIndex+1])
            words.pop(wordIndex)
            words.pop(wordIndex)
            words.insert(wordIndex, time)
        #7:00:00 || 7:00 || 7:00.
        elif(isTimeFormat(words[wordIndex]) or isTimeFormat(words[wordIndex][:-1])):
            if(isTimeFormat(words[wordIndex])):
                time = isTime(words[wordIndex])
                words.pop(wordIndex)
                words.insert(wordIndex, time)
            else:
                punc = words[wordIndex][-1]
                time = isTime(words[wordIndex][:-1])
                words.pop(wordIndex)
                words.insert(wordIndex, time)
                words.insert(wordIndex+1, punc)
        
        wordIndex += 1
    
    wordIndex = 0
    while (wordIndex < len(words)) :
        tempList = containsClitic(words[wordIndex])
        if(len(tempList) > 1):
            words.pop(wordIndex)
            words[wordIndex:wordIndex] = tempList
            wordIndex += len(tempList) - 1
        wordIndex += 1
     
    wordIndex = 0
    while wordIndex < len(words) :
        if(isUrl(words[wordIndex]) == False):
            tempList = containsPunc(words[wordIndex])
            if(len(tempList) > 1):
                words.pop(wordIndex)
                words[wordIndex:wordIndex] = tempList
                wordIndex += len(tempList) - 1
        wordIndex += 1
    
    wordIndex = 0
    while wordIndex < len(words) :
        finalDate = ""
        if(wordIndex+1 < len(words) and words[wordIndex+1] == "'" and words[wordIndex] in months):
            month = months[words[wordIndex]]
            finalDate += month
            if(wordIndex+2 < len(words) and isNumberinWord(words[wordIndex+2])):
                finalDate += "-"
                for letter in words[wordIndex+2]:
                    if(letter.isdigit()):
                        finalDate += letter
            date = convertDate(finalDate)
            words.pop(wordIndex)
            words.pop(wordIndex)
            words.pop(wordIndex)
            words.insert(wordIndex, date)
        elif(words[wordIndex] in months):
            month = months[words[wordIndex]]
            if(wordIndex-1 >= 0 and isNumberinWord(words[wordIndex-1]) and isFinalDate(words[wordIndex-1]) == False):
                for letter in words[wordIndex-1]:
                    if(letter.isdigit()):
                        finalDate += letter
                finalDate += "-" + month + "-"
                if(wordIndex+1 < len(words)):
                    if(re.findall("[\.!?:,;\'\"\)\(\]-]", words[wordIndex+1])):
                        if(wordIndex+2 < len(words)):
                            for letter in words[wordIndex+2]:
                                if(letter.isdigit()):
                                    finalDate += letter
                        date = convertDate(finalDate)
                        words.pop(wordIndex-1)
                        words.pop(wordIndex-1)
                        words.pop(wordIndex-1)
                        words.pop(wordIndex-1)
                        words.insert(wordIndex-1, date)
                    else:
                        lengthNum = 0
                        isPureNumber = 0
                        for letter in words[wordIndex+1]:
                            if(letter.isdigit()):
                                finalDate += letter
                                lengthNum += 1
                                isPureNumber = 1
                            else:
                                isPureNumber = 0
                                break
                        date = convertDate(finalDate)
                        if((lengthNum == 2 or lengthNum == 4) and isPureNumber == 1):
                            words.pop(wordIndex-1)
                            words.pop(wordIndex-1)
                            words.pop(wordIndex-1)
                            words.insert(wordIndex-1, date)
                        else:
                            words.pop(wordIndex-1)
                            words.pop(wordIndex-1)
                            words.insert(wordIndex-1, date)
                        
                else:
                    date = convertDate(finalDate)
                    words.pop(wordIndex-1)
                    words.pop(wordIndex-1)
                    words.insert(wordIndex-1, date)
            elif(wordIndex+1 < len(words) and isNumberinWord(words[wordIndex+1])):
                finalDate = month +"-"
                for letter in words[wordIndex+1]:
                    if(letter.isdigit()):
                        finalDate += letter
                date = convertDate(finalDate)
                words.pop(wordIndex)
                words.pop(wordIndex)
                words.insert(wordIndex, date)
            else:
                finalDate = month
                date = convertDate(finalDate)
                words[wordIndex] = date
        wordIndex += 1
    
    
    tokens.append(words)
#print(tokens)

In [366]:
import datetime
def isDate(word):
    prefix = "CF:D:"
    formatList = [
                  '%d.%m.%y', '%d.%m.%Y',
                  '%d/%m/%y', '%d/%m/%Y',
                  '%d-%m-%y','%d-%m-%Y',
                  '%m.%d.%y', '%m.%d.%Y',
                  '%m/%d/%y', '%m/%d/%Y',
                  '%m-%d-%y','%m-%d-%Y',
                  '%y.%m.%d', '%Y.%m.%d',
                  '%y/%m/%d', '%Y/%m/%d',
                  '%y-%m-%d', '%Y-%m-%d',
                 ]
    
    for fmt in formatList:
        try:
            val = datetime.datetime.strptime(word,fmt)
            date = str(val).split()
            return str(prefix+date[0])
        except ValueError:
            pass
    return None

#checks if it is a time
def isTimeFormat(word):
    formatList = [
                  "%I:%M %p",
                    "%I:%M",
                    "%I %p",
                    "%I:%M:%S %p",
                    "%I:%M:%S"
                 ]
    
    for fmt in formatList:
        try:
            datetime.datetime.strptime(word,fmt)
            return True
        except ValueError:
            pass
    return False

#I = hour, M = min, S = sec, p = AM/PM 
def isTime(word):
    prefix = "CF:T:"
    formatList = [
                  "%I:%M %p",
                    "%I:%M",
                    "%I %p",
                    "%I:%M:%S %p",
                    "%I:%M:%S"
                 ]
    
    for fmt in formatList:
        try:
            in_time = datetime.datetime.strptime(word,fmt)
            out_time = datetime.datetime.strftime(in_time, "%H:%M")
            return str(prefix+out_time)
        except ValueError:
            pass
    return None

In [367]:
def isUrl(word):
    urlRegex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))" 
    if(re.findall(urlRegex, word)):
        return True
    else:
        return False

In [368]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it's": "it is",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

def containsClitic(word):
    tempList = []
    contraction = ""
    if word.lower() in contractions:
        contraction =  contractions[word.lower()]
        if(word.isupper()):
            contraction = contraction.upper()
        elif(word[0].isupper()):
            contraction = contraction[0].upper() + contraction[1:]
    if(len(contraction) != 0):
        tempList = contraction.split()
    return tempList
       
#print(containsClitic("i'm")) #HAD NOT

In [369]:
def containsPunc(word):
    tempList = []
    positions = re.findall("[\.!?:,;\'\"\)\(\]-]", word)
    if(word.startswith("CF:T:")):
        tempList.append(word)
        return tempList
    for pos in positions:
        tempStr = ""
        i = 0
        for letter in word:
            i += 1
            if(letter != pos):
                tempStr += letter 
            else:
                if(letter == "'"):
                    idx = word.index(letter)
                    #apostrophe should be last letter of word and before apostrophe there should be s
                    if(idx == len(word) - 1 and (idx - 1) >= 0 and word[idx-1] == "s"):
                        tempList.append(tempStr)
                        tempList.append("'s")
                        word = word[idx+1:]
                    elif(idx+1 < len(word) and word[idx+1] == "s" and (idx+2 == len(word) or word[idx+2].isalpha() == False)):
                        if(len(tempStr) != 0):
                            tempList.append(tempStr)
                        tempStr = "'s"
                        tempList.append(tempStr)
                        word = word[idx+2:]
                    else:
                        if(len(tempStr) != 0):
                            tempList.append(tempStr)
                        tempList.append(pos)
                        word = word[i:]
                    
                elif(len(tempStr) != 0):
                    tempList.append(tempStr)
                    tempList.append(pos)
                    word = word[i:]
                
                elif(letter == pos and len(tempStr) == 0):
                    tempList.append(pos)
                    word = word[i:]
                break
    if(len(word) != 0):
        tempList.append(word)
    return tempList

print(containsPunc("'This is Jones'"))

["'", 'This is Jones', "'s"]


In [370]:
outputFile = "output.txt"

try:
    with open(outputFile, 'w') as fp:
       i = 0
       for tweet in tweets :
            fp.write(tweet)
            fp.write(str(len(tokens[i])))
            fp.write("\n")
            for token in tokens[i] :
                temp = token + "\n"
                fp.write(temp)
            i += 1
except IOError as e:
    print("Couldn't open or write to file (%s)." % e)
    

In [ ]:
emojis = 
    [":@", ":-@",
">:o",
">:0",
"D:<",
"D:",
"D8",
"D;",
"D=",
"Dx",
">.<",
">_<",
"d:<",
"d:",
"d8",
"d;",
"d=",
"dx",
"v.v",
":/",
":\"",
"=/",
"=\"",
">:/",
">:\",
":-/",
":-\"",
":)",
"(:",
";)",
";(",
"(;",
");",
":-)",
":3",
":d",
":D",
"xd",
":')",
"^_^",
"^.^",
":]",
":}",
":p",
":b",
"=p",
"=b",
":-p",
":-b",
"=)",
":(",
"):",
":'(",
":c",
":-(",
"</3",
":[",
":{",
"T.T",
"o_o",
"O_O",
"0_o",
"o_0",
"0_O",
"O_0",
"o.o",
"O.O",
"0.o",
"o.0",
":o",
":-o",
"<3",
":p",
":b",
"=p",
"=b",
":-p",
":-b",
":$"
]
